<a href="https://colab.research.google.com/github/yanxingjianken/DURF/blob/main/YOLOv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def check_installed_libraries():
    libraries = ['cv2', 'numpy', 'pydrive', 'tensorflow']

    for library in libraries:
        try:
            exec(f"import {library}")
            print(f"{library} has been installed.")
        except ImportError:
            print(f"{library} has not been installed.")

check_installed_libraries()


cv2 has been installed.
numpy has been installed.
pydrive has been installed.
tensorflow has been installed.


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving client_secret_1024937029988-qom8cb34j0o9q3sqmlpt3g8p1i7g80ik.apps.googleusercontent.com.json to client_secret_1024937029988-qom8cb34j0o9q3sqmlpt3g8p1i7g80ik.apps.googleusercontent.com.json
User uploaded file "client_secret_1024937029988-qom8cb34j0o9q3sqmlpt3g8p1i7g80ik.apps.googleusercontent.com.json" with length 401 bytes


In [ ]:
import os
from google.colab import drive
import cv2
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Specify the directory of your videos
dir_path = "/content/drive/MyDrive/NYU/2023_Spring/durf/traffic data"

# Get a list of all .mp4 files
file_list = [f for f in os.listdir(dir_path) if f.endswith('.mp4')]

# Process each video
for file_name in file_list:
    # Open the video file
    cap = cv2.VideoCapture(os.path.join(dir_path, file_name))

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file")
        continue

    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:
            # Convert to bird's eye view
            src = np.float32([[0, 0], [0, frame.shape[0]], [frame.shape[1], 0], [frame.shape[1], frame.shape[0]]])
            dst = np.float32([[0, 0], [0, 500], [800, 0], [800, 500]])  # You need to adjust these points

            matrix = cv2.getPerspectiveTransform(src, dst)
            bird_view = cv2.warpPerspective(frame, matrix, (800, 500))

            cv2.imshow('Frame', bird_view)

            # Press Q on keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        # Break the loop
        else:
            break

    # When everything done, release the video capture object
    cap.release()

# Closes all the frames
cv2.destroyAllWindows()


MessageError: ignored

In [ ]:
import cv2
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import datetime

# Load the trained model
net = cv2.dnn.readNet('yolov7.weights', 'yolov7.cfg')

# Connect to Google Drive
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# The folder id in Google Drive
folder_id = 'folder_id'  # Replace with your folder id

# Get the list of all files in directory
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()

# Prepare a function to process each video file
def process_video(video_file):
    cap = cv2.VideoCapture(video_file)
    ret, frame1 = cap.read()
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    speeds_and_densities = []

    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        # Speed
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        speed = np.mean(mag)

        # Density
        blob = cv2.dnn.blobFromImage(frame2, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward()
        car_area = sum([out[2]*out[3] for out in outs if out[1] == 'car'])
        density = car_area / (frame_width * frame_height)

        # Append to results
        speeds_and_densities.append((speed, density, cap.get(cv2.CAP_PROP_POS_MSEC)))

        prvs = next

    cap.release()

    # Convert to numpy array and save
    speeds_and_densities = np.array(speeds_and_densities)
    np.save(f'{os.path.splitext(video_file)[0]}.npy', speeds_and_densities)

# Process each video
for file in file_list:
    # Download file
    file.GetContentFile(file['title'])

    # Process the video
    process_video(file['title'])
